In [18]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration
import random
import json
from tqdm import tqdm
import time
import re
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import pandas as pd
import numpy as np
import os

In [31]:
import gc
import torch
import psutil

def clear_memory():
    # Clear CUDA cache if available
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.reset_accumulated_memory_stats()

    # Clear MPS cache if available
    if hasattr(torch.mps, 'empty_cache'):
        torch.mps.empty_cache()

    # Run garbage collection
    gc.collect()

def print_memory_stats():
    print("GPU Memory Stats:")
    if torch.cuda.is_available():
        print(f"CUDA memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
        print(f"CUDA memory cached: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
    elif hasattr(torch.mps, 'current_allocated_memory'):
        print(f"MPS memory allocated: {torch.mps.current_allocated_memory() / 1e9:.2f} GB")
        print(f"MPS memory cached: {torch.mps.driver_allocated_memory() / 1e9:.2f} GB")
    else:
        print("No GPU available")

    print("\nSystem Memory Stats:")
    vm = psutil.virtual_memory()
    print(f"Total: {vm.total / 1e9:.2f} GB")
    print(f"Available: {vm.available / 1e9:.2f} GB")
    print(f"Used: {vm.used / 1e9:.2f} GB")
    print(f"Percentage: {vm.percent}%")

if __name__ == "__main__":
    print("Memory stats before clearing:")
    print_memory_stats()

    print("\nClearing memory...")
    clear_memory()

    print("\nMemory stats after clearing:")
    print_memory_stats()

    print("\nIf you're still seeing high memory usage, consider restarting your Python environment.")

Memory stats before clearing:
GPU Memory Stats:
MPS memory allocated: 0.00 GB
MPS memory cached: 17.56 GB

System Memory Stats:
Total: 17.18 GB
Available: 2.90 GB
Used: 5.78 GB
Percentage: 83.1%

Clearing memory...

Memory stats after clearing:
GPU Memory Stats:
MPS memory allocated: 0.00 GB
MPS memory cached: 0.14 GB

System Memory Stats:
Total: 17.18 GB
Available: 7.71 GB
Used: 6.00 GB
Percentage: 55.1%

If you're still seeing high memory usage, consider restarting your Python environment.


In [19]:
# Set MPS memory efficiency options
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'  # Disable upper limit for memory allocations
torch.mps.set_per_process_memory_fraction(0.8) 

In [20]:
# Check for MPS (Metal Performance Shaders) availability
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Metal Performance Shaders) device")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA device")
else:
    device = torch.device("cpu")
    print("Using CPU device")

Using MPS (Metal Performance Shaders) device


In [32]:
def print_memory_usage():
    if hasattr(torch.mps, 'current_allocated_memory'):
        print(f"MPS memory allocated: {torch.mps.current_allocated_memory() / 1e9:.2f} GB")
        print(f"MPS memory cached: {torch.mps.driver_allocated_memory() / 1e9:.2f} GB")
    print(f"System memory used: {psutil.virtual_memory().percent}%")

In [33]:
# Data Generation

def generate_network():
    layers = []
    input_shape = [random.randint(1, 256) for _ in range(random.randint(1, 3))]
    current_shape = input_shape.copy()

    for _ in range(random.randint(1, 5)):
        layer_type = random.choice(['Linear', 'Conv1d'])
        if layer_type == 'Linear':
            in_features = current_shape[-1]
            out_features = random.randint(1, 256)
            layers.append(f"nn.Linear(in_features={in_features}, out_features={out_features})")
            current_shape = current_shape[:-1] + [out_features]
        elif layer_type == 'Conv1d':
            in_channels = current_shape[0]
            out_channels = random.randint(1, 64)
            kernel_size = random.randint(1, 5)
            layers.append(f"nn.Conv1d(in_channels={in_channels}, out_channels={out_channels}, kernel_size={kernel_size})")
            current_shape[0] = out_channels

    return input_shape, current_shape, layers

def generate_description(input_shape, output_shape):
    return f"This neural network takes an input of shape {input_shape} and produces an output of shape {output_shape}."

def generate_data(num_samples, time_limit):
    data = []
    start_time = time.time()
    for _ in tqdm(range(num_samples), desc="Generating data"):
        if time.time() - start_time > time_limit:
            break
        input_shape, output_shape, layers = generate_network()
        network = "nn.Sequential(\n    " + ",\n    ".join(layers) + "\n)"
        description = generate_description(input_shape, output_shape)
        data.append({"network": network, "description": description})
    return data

In [34]:
# Data Analysis and Visualization

def analyze_data(data):
    layer_types = []
    input_shapes = []
    output_shapes = []

    for item in data:
        network = item['network']
        layer_types.extend(re.findall(r'nn\.(\w+)', network))
        input_shape, output_shape = extract_shapes(item['description'])
        input_shapes.append(eval(input_shape))
        output_shapes.append(eval(output_shape))

    return layer_types, input_shapes, output_shapes

In [35]:
def visualize_data(layer_types, input_shapes, output_shapes):
    plt.figure(figsize=(12, 6))
    sns.countplot(y=layer_types)
    plt.title('Distribution of Layer Types')
    plt.tight_layout()
    plt.savefig('layer_types_distribution.png')
    plt.close()

    input_dims = [len(shape) for shape in input_shapes]
    plt.figure(figsize=(8, 6))
    sns.histplot(input_dims, bins=range(1, max(input_dims)+2), kde=True)
    plt.title('Distribution of Input Shape Dimensions')
    plt.xlabel('Number of Dimensions')
    plt.tight_layout()
    plt.savefig('input_shape_dimensions.png')
    plt.close()

    output_dims = [len(shape) for shape in output_shapes]
    plt.figure(figsize=(8, 6))
    sns.histplot(output_dims, bins=range(1, max(output_dims)+2), kde=True)
    plt.title('Distribution of Output Shape Dimensions')
    plt.xlabel('Number of Dimensions')
    plt.tight_layout()
    plt.savefig('output_shape_dimensions.png')
    plt.close()

    input_sizes = [np.prod(shape) for shape in input_shapes]
    plt.figure(figsize=(10, 6))
    sns.histplot(input_sizes, bins=30, kde=True)
    plt.title('Distribution of Input Sizes')
    plt.xlabel('Input Size (Total Number of Elements)')
    plt.xscale('log')
    plt.tight_layout()
    plt.savefig('input_sizes_distribution.png')
    plt.close()

    output_sizes = [np.prod(shape) for shape in output_shapes]
    plt.figure(figsize=(10, 6))
    sns.histplot(output_sizes, bins=30, kde=True)
    plt.title('Distribution of Output Sizes')
    plt.xlabel('Output Size (Total Number of Elements)')
    plt.xscale('log')
    plt.tight_layout()
    plt.savefig('output_sizes_distribution.png')
    plt.close()

In [36]:
# Model and Dataset

class NeuralNetworkDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_encoding = self.tokenizer.encode_plus(
            item['network'],
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        target_encoding = self.tokenizer.encode_plus(
            item['description'],
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': input_encoding['input_ids'].flatten(),
            'attention_mask': input_encoding['attention_mask'].flatten(),
            'labels': target_encoding['input_ids'].flatten()
        }

In [37]:
# Training and Evaluation

def train(model, train_dataloader, optimizer, device, max_time, accumulation_steps=4):
    model.train()
    total_loss = 0
    start_time = time.time()
    optimizer.zero_grad()
    for i, batch in enumerate(tqdm(train_dataloader, desc="Training")):
        if time.time() - start_time > max_time:
            break
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Normalize loss to account for accumulation
        loss = loss / accumulation_steps
        loss.backward()

        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

    return total_loss / len(train_dataloader)

In [38]:
def evaluate(model, eval_dataloader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

    return total_loss / len(eval_dataloader)

In [39]:
def generate_description_for_network(model, tokenizer, network_str, device):
    input_ids = tokenizer.encode(network_str, return_tensors="pt").to(device)
    output = model.generate(input_ids, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def extract_shapes(description):
    shapes = re.findall(r'\[.*?\]', description)
    return shapes[0] if shapes else None, shapes[-1] if shapes else None

In [40]:
def evaluate_model(model, tokenizer, test_data, device):
    model.eval()
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for item in tqdm(test_data, desc="Evaluating"):
            input_text = item['network']
            true_description = item['description']
            
            input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
            output = model.generate(input_ids, max_length=100)
            predicted_description = tokenizer.decode(output[0], skip_special_tokens=True)
            
            true_input, true_output = extract_shapes(true_description)
            pred_input, pred_output = extract_shapes(predicted_description)
            
            true_labels.append((true_input, true_output))
            predicted_labels.append((pred_input, pred_output))

    return true_labels, predicted_labels

In [41]:
def calculate_metrics(true_labels, predicted_labels):
    correct_input = sum(t[0] == p[0] for t, p in zip(true_labels, predicted_labels))
    correct_output = sum(t[1] == p[1] for t, p in zip(true_labels, predicted_labels))
    correct_both = sum(t == p for t, p in zip(true_labels, predicted_labels))
    
    total = len(true_labels)
    
    accuracy = correct_both / total
    
    # Convert to binary classification for F1 score
    y_true = [1 if t == p else 0 for t, p in zip(true_labels, predicted_labels)]
    y_pred = [1 if t == p else 0 for t, p in zip(true_labels, predicted_labels)]
    
    f1 = f1_score(y_true, y_pred)
    
    return {
        'Accuracy': accuracy,
        'F1 Score': f1
    }

In [43]:
# Main execution
def main():
    # Check for MPS (Metal Performance Shaders) availability
    if torch.backends.mps.is_available():
        device = torch.device("mps")
        print("Using MPS (Metal Performance Shaders) device")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
        print("Using CUDA device")
    else:
        device = torch.device("cpu")
        print("Using CPU device")
        print("Starting the 3D Constrained Neural Network Description Generator...")
    
    # Data Generation
    print("Generating data...")
    train_data = generate_data(50000, 7200)  # 2 hours = 7200 seconds
    eval_data = generate_data(5000, 600)  # 10 minutes for eval data

    print(f"Generated {len(train_data)} training samples and {len(eval_data)} evaluation samples.")

    # Save data to files
    with open('train_data.json', 'w') as f:
        json.dump(train_data, f)
    with open('eval_data.json', 'w') as f:
        json.dump(eval_data, f)

    print("Data generation completed and saved.")

    # Data Analysis and Visualization
    print("Analyzing and visualizing data...")
    layer_types, input_shapes, output_shapes = analyze_data(train_data)
    visualize_data(layer_types, input_shapes, output_shapes)

    # Model Training
    print("Preparing model and datasets...")
    tokenizer = T5Tokenizer.from_pretrained("t5-small")
    model = T5ForConditionalGeneration.from_pretrained("t5-small")

    train_dataset = NeuralNetworkDataset(train_data, tokenizer, max_length=128)
    eval_dataset = NeuralNetworkDataset(eval_data, tokenizer, max_length=128)
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    eval_dataloader = DataLoader(eval_dataset, batch_size=4)

    print("Starting training...")
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

    num_epochs = 5
    max_train_time = 7200  # 2 hours
    start_time = time.time()
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        train_loss = train(model, train_dataloader, optimizer, device, max_train_time - (time.time() - start_time))
        eval_loss = evaluate(model, eval_dataloader, device)
        print(f"Train loss: {train_loss:.4f}")
        print(f"Eval loss: {eval_loss:.4f}")
        # Save checkpoint after each epoch
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'eval_loss': eval_loss,
        }, f'checkpoint_epoch_{epoch+1}.pth')
        
        if time.time() - start_time > max_train_time:
            print("Training time limit reached. Stopping training.")
            break

    # Save the model
    model.save_pretrained("neural_network_description_model")
    tokenizer.save_pretrained("neural_network_description_model")

    print("Training completed and model saved.")
    # Model Evaluation
    print("Evaluating the model...")
    true_labels, predicted_labels = evaluate_model(model, tokenizer, eval_data, device)

    # Calculate and display metrics
    metrics = calculate_metrics(true_labels, predicted_labels)
    print("\nModel Performance Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

    # Evaluation on 5 different unique neural network architectures
    print("\nEvaluating on 5 different unique neural network architectures...")
    
    test_networks = [
        """nn.Sequential(
            nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=10)
        )""",
        """nn.Sequential(
            nn.Linear(in_features=100, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=1)
        )""",
        """nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5),
            nn.ReLU(),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=10)
        )""",
        """nn.Sequential(
            nn.Linear(in_features=784, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=10)
        )""",
        """nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3),
            nn.ReLU(),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=1)
        )"""
    ]
    true_shapes = [
        ([3, 100], [10]),
        ([100], [1]),
        ([1, 100], [10]),
        ([784], [10]),
        ([1, 100], [1])
    ]

    test_true_labels = []
    test_predicted_labels = []

    for i, network in enumerate(test_networks):
        print(f"\nTesting network {i+1}:")
        print(network)
        generated_description = generate_description_for_network(model, tokenizer, network, device)
        print("Generated description:", generated_description)
        pred_input, pred_output = extract_shapes(generated_description)
        print(f"Predicted shapes: Input {pred_input}, Output {pred_output}")
        print(f"True shapes: Input {true_shapes[i][0]}, Output {true_shapes[i][1]}")
        
        test_true_labels.append((str(true_shapes[i][0]), str(true_shapes[i][1])))
        test_predicted_labels.append((pred_input, pred_output))

    # Calculate metrics for the 5 test networks
    test_metrics = calculate_metrics(test_true_labels, test_predicted_labels)
    print("\nTest Network Performance Metrics:")
    for metric, value in test_metrics.items():
        print(f"{metric}: {value:.4f}")

    # Detailed analysis of test results
    print("\nDetailed analysis of test results:")
    for i, (true, pred) in enumerate(zip(test_true_labels, test_predicted_labels)):
        print(f"\nNetwork {i+1}:")
        print(f"True: Input {true[0]}, Output {true[1]}")
        print(f"Predicted: Input {pred[0]}, Output {pred[1]}")
        print(f"Input shape correct: {true[0] == pred[0]}")
        print(f"Output shape correct: {true[1] == pred[1]}")
        print(f"Both shapes correct: {true == pred}")

    # Additional Analysis
    print("\nAdditional Analysis:")
     # Distribution of correct predictions
    correct_predictions = sum(t == p for t, p in zip(test_true_labels, test_predicted_labels))
    print(f"Correct predictions: {correct_predictions} out of {len(test_true_labels)}")
    
    # Analysis of input shape predictions
    input_shape_correct = sum(t[0] == p[0] for t, p in zip(test_true_labels, test_predicted_labels))
    print(f"Correct input shape predictions: {input_shape_correct} out of {len(test_true_labels)}")
    
    # Analysis of output shape predictions
    output_shape_correct = sum(t[1] == p[1] for t, p in zip(test_true_labels, test_predicted_labels))
    print(f"Correct output shape predictions: {output_shape_correct} out of {len(test_true_labels)}")
    
    # Error analysis
    errors = [(i, t, p) for i, (t, p) in enumerate(zip(test_true_labels, test_predicted_labels)) if t != p]
    if errors:
        print("\nError Analysis:")
        for i, true, pred in errors:
            print(f"Network {i+1}:")
            print(f"  True: Input {true[0]}, Output {true[1]}")
            print(f"  Predicted: Input {pred[0]}, Output {pred[1]}")
    else:
        print("\nNo errors in test set predictions.")

    print("\n3D Constrained Neural Network Description Generator process completed.")

if __name__ == "__main__":
    main()

Using MPS (Metal Performance Shaders) device
Generating data...


Generating data: 100%|██████████| 5000/5000 [00:00<00:00, 176541.32it/s]


Generated 50000 training samples and 5000 evaluation samples.
Data generation completed and saved.
Analyzing and visualizing data...
Preparing model and datasets...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Starting training...
Epoch 1/5


...

Train loss: 0.0534
Eval loss: 0.0439
Epoch 5/5


Evaluating: 100%|██████████| 1250/1250 [00:36<00:00, 34.42it/s]


Train loss: 0.0497
Eval loss: 0.0421
Training completed and model saved.
Evaluating the model...


Evaluating:   0%|          | 0/5000 [00:00<?, ?it/s]


RuntimeError: Placeholder storage has not been allocated on MPS device!

In [44]:
def load_saved_model(checkpoint_path, device):
    """
    Load a saved T5 model from a checkpoint.
    
    Args:
    checkpoint_path (str): Path to the saved checkpoint file.
    device (torch.device): The device to load the model onto.
    
    Returns:
    model (T5ForConditionalGeneration): The loaded model.
    tokenizer (T5Tokenizer): The tokenizer associated with the model.
    optimizer (torch.optim.Optimizer): The optimizer (if saved in the checkpoint).
    start_epoch (int): The epoch to resume training from.
    """
    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    
    # Initialize the model and tokenizer
    model = T5ForConditionalGeneration.from_pretrained("t5-small")
    tokenizer = T5Tokenizer.from_pretrained("t5-small")
    
    # Load the model state dict
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    
    # Initialize the optimizer (AdamW in this case, but adjust if you used a different optimizer)
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    
    # Load the optimizer state dict if it exists in the checkpoint
    if 'optimizer_state_dict' in checkpoint:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    # Get the epoch number to resume training
    start_epoch = checkpoint.get('epoch', 0) + 1
    
    print(f"Loaded model from epoch {start_epoch - 1}")
    print(f"Train loss at save point: {checkpoint.get('train_loss', 'N/A')}")
    
    return model, tokenizer, optimizer, start_epoch

In [45]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
checkpoint_path = "checkpoint_epoch_5.pth"  # Adjust this to your checkpoint file path
model, tokenizer, optimizer, start_epoch = load_saved_model(checkpoint_path, device) 
print(f"Model loaded successfully and moved to {device}")
print(f"Ready to resume training from epoch {start_epoch}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded model from epoch 4
Train loss at save point: 0.049687410358786585
Model loaded successfully and moved to mps
Ready to resume training from epoch 5


In [53]:
device = torch.device("cpu")
model = model.to(device)
eval_data = generate_data(5000, 600)  # 10 minutes for eval data
print("Training completed and model saved.")
    # Model Evaluation
print("Evaluating the model...")
true_labels, predicted_labels = evaluate_model(model, tokenizer, eval_data, device)

# Calculate and display metrics
metrics = calculate_metrics(true_labels, predicted_labels)
print("\nModel Performance Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

Generating data: 100%|██████████| 5000/5000 [00:00<00:00, 158579.62it/s]


Training completed and model saved.
Evaluating the model...


Evaluating: 100%|██████████| 5000/5000 [46:12<00:00,  1.80it/s]


Model Performance Metrics:
Accuracy: 0.4496
F1 Score: 1.0000


In [57]:
# Evaluation on 5 different unique neural network architectures
print("\nEvaluating on 5 different unique neural network architectures...")
    
test_networks = [
        """nn.Sequential(
            nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=10)
        )""",
        """nn.Sequential(
            nn.Linear(in_features=100, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=1)
        )""",
        """nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5),
            nn.ReLU(),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=10)
        )""",
        """nn.Sequential(
            nn.Linear(in_features=784, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=10)
        )""",
        """nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3),
            nn.ReLU(),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=1)
        )"""
    ]


Evaluating on 5 different unique neural network architectures...


In [58]:
true_shapes = [
        ([3, 100], [10]),
        ([100], [1]),
        ([1, 100], [10]),
        ([784], [10]),
        ([1, 100], [1])
    ]

test_true_labels = []
test_predicted_labels = []

for i, network in enumerate(test_networks):
    print(f"\nTesting network {i+1}:")
    print(network)
    generated_description = generate_description_for_network(model, tokenizer, network, device)
    print("Generated description:", generated_description)
    pred_input, pred_output = extract_shapes(generated_description)
    print(f"Predicted shapes: Input {pred_input}, Output {pred_output}")
    print(f"True shapes: Input {true_shapes[i][0]}, Output {true_shapes[i][1]}")
        
    test_true_labels.append((str(true_shapes[i][0]), str(true_shapes[i][1])))
    test_predicted_labels.append((pred_input, pred_output))


Testing network 1:
nn.Sequential(
            nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=10)
        )
Generated description: This neural network takes an input of shape [3, 125, 128] and produces an output of shape [128, 125, 10].
Predicted shapes: Input [3, 125, 128], Output [128, 125, 10]
True shapes: Input [3, 100], Output [10]

Testing network 2:
nn.Sequential(
            nn.Linear(in_features=100, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=1)
        )
Generated description: This neural network takes an input of shape [100] and produces an output of shape [1].
Predicted shapes: Input [100], Output [1]
True shapes: Input [100], Output [1]

Testing network 3:
nn.Sequenti

In [59]:
# Calculate metrics for the 5 test networks
test_metrics = calculate_metrics(test_true_labels, test_predicted_labels)
print("\nTest Network Performance Metrics:")
for metric, value in test_metrics.items():
    print(f"{metric}: {value:.4f}")

# Detailed analysis of test results
print("\nDetailed analysis of test results:")
for i, (true, pred) in enumerate(zip(test_true_labels, test_predicted_labels)):
    print(f"\nNetwork {i+1}:")
    print(f"True: Input {true[0]}, Output {true[1]}")
    print(f"Predicted: Input {pred[0]}, Output {pred[1]}")
    print(f"Input shape correct: {true[0] == pred[0]}")
    print(f"Output shape correct: {true[1] == pred[1]}")
    print(f"Both shapes correct: {true == pred}")

# Additional Analysis
print("\nAdditional Analysis:")
# Distribution of correct predictions
correct_predictions = sum(t == p for t, p in zip(test_true_labels, test_predicted_labels))
print(f"Correct predictions: {correct_predictions} out of {len(test_true_labels)}")
    
# Analysis of input shape predictions
input_shape_correct = sum(t[0] == p[0] for t, p in zip(test_true_labels, test_predicted_labels))
print(f"Correct input shape predictions: {input_shape_correct} out of {len(test_true_labels)}")
    
# Analysis of output shape predictions
output_shape_correct = sum(t[1] == p[1] for t, p in zip(test_true_labels, test_predicted_labels))
print(f"Correct output shape predictions: {output_shape_correct} out of {len(test_true_labels)}")


Test Network Performance Metrics:
Accuracy: 0.4000
F1 Score: 1.0000

Detailed analysis of test results:

Network 1:
True: Input [3, 100], Output [10]
Predicted: Input [3, 125, 128], Output [128, 125, 10]
Input shape correct: False
Output shape correct: False
Both shapes correct: False

Network 2:
True: Input [100], Output [1]
Predicted: Input [100], Output [1]
Input shape correct: True
Output shape correct: True
Both shapes correct: True

Network 3:
True: Input [1, 100], Output [10]
Predicted: Input [1, 62, 64], Output [64, 62, 10]
Input shape correct: False
Output shape correct: False
Both shapes correct: False

Network 4:
True: Input [784], Output [10]
Predicted: Input [784], Output [10]
Input shape correct: True
Output shape correct: True
Both shapes correct: True

Network 5:
True: Input [1, 100], Output [1]
Predicted: Input [1, 62, 64], Output [64, 62, 1]
Input shape correct: False
Output shape correct: False
Both shapes correct: False

Additional Analysis:
Correct predictions: 2 

In [61]:
# Error analysis
errors = [(i, t, p) for i, (t, p) in enumerate(zip(test_true_labels, test_predicted_labels)) if t != p]
if errors:
    print("\nError Analysis:")
    for i, true, pred in errors:
        print(f"Network {i+1}:")
        print(f"  True: Input {true[0]}, Output {true[1]}")
        print(f"  Predicted: Input {pred[0]}, Output {pred[1]}")
else:
    print("\nNo errors in test set predictions.")

print("\n3D Constrained Neural Network Description Generator process completed.")


Error Analysis:
Network 1:
  True: Input [3, 100], Output [10]
  Predicted: Input [3, 125, 128], Output [128, 125, 10]
Network 3:
  True: Input [1, 100], Output [10]
  Predicted: Input [1, 62, 64], Output [64, 62, 10]
Network 5:
  True: Input [1, 100], Output [1]
  Predicted: Input [1, 62, 64], Output [64, 62, 1]

3D Constrained Neural Network Description Generator process completed.
